# 重新定义大模型和智能体

## 综合例子

In [1]:
# 创建一个工具
from illufly.chat import ChatQwen
from illufly.types import ToolAgent, Template, EventBlock

def get_current_weather(location: str):
    """获取城市的天气情况"""
    # return f"{location}今天是晴天。 "
    yield EventBlock("chunk", f"{location}今天是晴天。 ")

## 声明大模型
qwen = ChatQwen(
    tools=[ToolAgent(get_current_weather)],
    knowledge=["这里的兔子是指嫦娥的宠物兔子"],
    memory=[("system", "你是一个作家")]
)

qwen("先查询广州天气，然后结合兔子的主人，帮我写一首关于兔子的四句打油诗", verbose=True)

[INFO] 记住 10 轮对话
[TOOLS_CALL_CHUNK] {"index": 0, "id": "call_caa7b381f4d94aee9d1e3a", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "广州\"}"}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}

[USAGE] {"input_tokens": 207, "output_tokens": 18, "total_tokens": 225}

[TOOLS_CALL_FINAL] [{"index": 0, "id": "call_caa7b381f4d94aee9d1e3a", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}]
广州今天是晴天。 
[TOOL_RESP_FINAL] 广州今天是晴天。 
在广州的晴朗日子下，
小白兔你在何方？
作为嫦娥的小伙伴，
人间的天气你可曾想往？

虽然天上不会变，
但愿你的日子如人间晴朗。
这里向你遥寄祝愿，
小白兔，愿你永远欢畅。 

请注意，这首打油诗虽然是基于

"在广州的晴朗日子下，\n小白兔你在何方？\n作为嫦娥的小伙伴，\n人间的天气你可曾想往？\n\n虽然天上不会变，\n但愿你的日子如人间晴朗。\n这里向你遥寄祝愿，\n小白兔，愿你永远欢畅。 \n\n请注意，这首打油诗虽然是基于您的要求创作的，但其中'小白兔'只是代表了嫦娥的宠物兔子，并非真的在地球或广州。毕竟，我们的小兔子实际上是居住在月宫中的。"

In [2]:
qwen.memory

[{'role': 'system', 'content': '你是一个作家'},
 {'role': 'user', 'content': '先查询广州天气，然后结合兔子的主人，帮我写一首关于兔子的四句打油诗'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_caa7b381f4d94aee9d1e3a',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'tool_call_id': 'call_caa7b381f4d94aee9d1e3a',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '广州今天是晴天。 '},
 {'role': 'assistant',
  'content': "在广州的晴朗日子下，\n小白兔你在何方？\n作为嫦娥的小伙伴，\n人间的天气你可曾想往？\n\n虽然天上不会变，\n但愿你的日子如人间晴朗。\n这里向你遥寄祝愿，\n小白兔，愿你永远欢畅。 \n\n请注意，这首打油诗虽然是基于您的要求创作的，但其中'小白兔'只是代表了嫦娥的宠物兔子，并非真的在地球或广州。毕竟，我们的小兔子实际上是居住在月宫中的。"}]

## 多轮对话

使用`messages`管理多轮对话的记忆。<br>
你也可以在`chat`函数中通过参数`k`限制希望保留的对话轮次，但这不影响`messages`记录完整的对话过程。

In [1]:
from illufly.chat import FakeLLM, ChatZhipu, ChatQwen
from illufly.io import log, alog

a = ChatQwen()

In [2]:
a.get_chat_memory(1)

[]

## 回调工具

In [5]:
from illufly.chat import ChatQwen
from illufly.types import ToolAgent

def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

qwen = ChatQwen(tools=[ToolAgent(get_current_weather)])
qwen("今天广州天气如何啊")



广州今天是晴天。 
广州今天的天气是晴天。


'广州今天的天气是晴天。'

In [6]:
qwen.tools[0].tool_desc

{'type': 'function',
 'function': {'name': 'get_current_weather',
  'description': '获取城市的天气情况',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string', 'description': ''}},
   'required': ['location']}}}

In [11]:
qwen("这是怎么得出来的？")

我是通过使用天气获取插件查询得到的。我询问了广州今天的天气，插件返回的结果是晴天。


'我是通过使用天气获取插件查询得到的。我询问了广州今天的天气，插件返回的结果是晴天。'

In [12]:
qwen("再看看上海")



上海今天是晴天。 
上海今天是晴天。


'上海今天是晴天。'

## 何时构建新对话

### 指定新对话

### 通过携带 system 角色的消息列表，发起新对话